# Cross-Layer Equalization (CLE) and Bias Correction (BC)

This notebook showcases a working code example of how to use AIMET to apply Cross-Layer Equalization (CLE) and Bias Correction (BC). CLE and BC are post-training quantization techniques that aim to improve quantized accuracy of a given model. CLE does not need any data samples. BC may optionally need unlabelled data samples. These techniques help recover quantized accuracy when the model quantization is sensitive to parameter quantization as opposed to activation quantization.

To learn more about this techniques, please refer to the "Data-Free Quantization Through Weight Equalization and Bias Correction" paper from ICCV 2019 - https://arxiv.org/abs/1906.04721

**Cross-Layer Equalization**
AIMET performs the following steps when running CLE:
1. Batch Norm Folding: Folds BN layers into Conv layers immediate before or after the Conv layers.
2. Cross-Layer Scaling: Given a set of consecutive Conv layers, equalizes the range of tensor values per-channel by scaling up/down per-channel weight tensor values of a layer and corresponding scaling down/up per-channel weight tensor values of the subsequent layer.
3. High Bias Folding: Cross-layer scaling may result in high bias parameter values for some layers. This technique folds some of the bias of a layer into the subsequent layer's parameters.

**Bias Correction**  
Quantization sometimes leads to a shift in layer outputs. This techniques helps correct this shift by adjusting the bias parameters of that layer. Note that this technique is generally applied after CLE, but it is a optional step.


#### Overall flow
This notebook covers the following
1. Instantiate the example evaluation and training pipeline
2. Load the FP32 model and evaluate the model to find the baseline FP32 accuracy
3. Create a quantization simulation model (with fake quantization ops inserted) and evaluate this simuation model to get a quantized accuracy score
4. Apply CLE, BC and and evaluate the simulation model to get a post-finetuned quantized accuracy score


#### What this notebook is not
* This notebook is not designed to show state-of-the-art results. For example, it uses a relatively quantization-friendly model like Resnet18. Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly.


---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification. If you already have a version of the dataset readily available, please use that. Else, please download the dataset from appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The ImageNet dataset typically has the following characteristics and the dataloader provided in this example notebook rely on these
- Subfolders 'train' for the training samples and 'val' for the validation samples. Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset. E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. But for the purpose of running this notebook, you could perhaps reduce the dataset to say 2 samples per class. This exercise is left upto the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [1]:
DATASET_DIR = '/teamspace/studios/this_studio/imagenet-val'         # Please replace this with a real directory

---

## 1. Example evaluation and training pipeline

The following is an example training and validation loop for this image classification task.

- **Does AIMET have any limitations on how the training, validation pipeline is written?** Not really. We will see later that AIMET will modify the user's model to create a QuantizationSim model which is still a PyTorch model. This QuantizationSim model can be used in place of the original model when doing inference or training.
- **Does AIMET put any limitation on the interface of the evaluate() or train() methods?** Not really. You should be able to use your existing evaluate and train routines as-is.


You are a Marketing profession specializing in marketing on social media applications such as linkedin. You are currently working in a startup that wants to market its expertise  in running deep learning models / AI on embedded devices

There was a repository for an application that performed Object Tracking using a finetuned Yolov8n on [dataset Name]. The object tracking was for UAV or Drone footage. I modified this repository to use be able to run on a Hardware accelerator (or NPU). 

I want to write a post for LinkedIn to market this while showing the performance improvement between running the same application, on a GTX 1050, CPU, and the Hardware accelerator. 

You are a Marketing profession specializing in marketing on social media applications such as linkedin. You are currently working in a startup that wants to market its expertise  in running deep learning models / AI on embedded devices

There is a Qualcomm Quantization framework known as Aimet which is used to quantize deep learning models while trying to minimize the loss in accuracy. I used one of their Post training quantization techniques (PTQ) 

In [2]:
import sys
sys.path.append("/teamspace/studios/this_studio/aimet/")

In [3]:
import os
import torch
from Examples.common import image_net_config
from Examples.torch.utils.image_net_evaluator import ImageNetEvaluator
from Examples.torch.utils.image_net_trainer import ImageNetTrainer
from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

class ImageNetDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(DATASET_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         is_training=False,
                                         num_workers=image_net_config.evaluation['num_workers']).data_loader
        return data_loader
    
    @staticmethod
    def get_train_dataloader() -> torch.utils.data.DataLoader:
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(DATASET_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         is_training=True,
                                         num_workers=image_net_config.evaluation['num_workers']).data_loader
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param iterations: the number of batches to be used to evaluate the model. A value of 'None' means the model will be
                           evaluated on the entire dataset once.
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = ImageNetEvaluator(DATASET_DIR, image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      num_workers=image_net_config.evaluation['num_workers'])

        return evaluator.evaluate(model, iterations=None, use_cuda=use_cuda)

---

## 2. Load the model and evaluate to get a baseline FP32 accuracy score

For this example notebook, we are going to load a pretrained resnet18 model from torchvision. Similarly, you can load any pretrained PyTorch model instead.

In [4]:
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
import copy

orig_model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)

In [5]:
# from torchvision.models import resnet18

# model = resnet18(pretrained=True)

AIMET quantization simulation requires the user's model definition to follow certain guidelines. For example, functionals defined in forward pass should be changed to equivalent torch.nn.Module.
AIMET user guide lists all these guidelines.
The following **ModelPreparer** API uses new graph transformation feature available in PyTorch 1.9+ version and automates model definition changes required to comply with the above guidelines. 

In [6]:
from aimet_torch.model_preparer import prepare_model
model = copy.deepcopy(orig_model)
model = prepare_model(model)

2024-09-11 10:27:19,660 - root - INFO - AIMET
2024-09-11 10:27:19,806 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.3.module_add} 
2024-09-11 10:27:19,807 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.5.module_add_1} 
2024-09-11 10:27:19,808 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.6.module_add_2} 
2024-09-11 10:27:19,809 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.8.module_add_3} 
2024-09-11 10:27:19,809 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.9.module_add_4} 
2024-09-11 10:27:19,810 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.10.module_add_5} 
2024-09-11 10:27:19,811 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.12.module_add_6} 
2024-09-11 10:27:19,812 - ModelPreparer - INFO - Functional         : Adding ne

---
We should decide whether to place the model on a CPU or CUDA device. This example code will use CUDA if available in your current execution environment. You can change this logic and force a device placement if needed.

In [7]:
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.to(torch.device('cuda'))
    
use_cuda

True

---
Let's determine the FP32 (floating point 32-bit) accuracy of this model using the evaluate() routine

In [8]:
accuracy = ImageNetDataPipeline.evaluate(model, use_cuda)
print(accuracy)

2024-09-11 10:27:26,843 - Dataloader - INFO - Dataset consists of 50000 images in 1000 classes
2024-09-11 10:27:26,846 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-09-11 10:27:26,846 - Eval - INFO - Evaluating nn.Module for 1563 iterations with batch_size 32


  0% (0 of 1563) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (4 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:02:34
  0% (5 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:03:45
  0% (6 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:04:53
  0% (7 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:05:08
  0% (8 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:05:15
  0% (9 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:05:36
  0% (10 of 1563) |                      | Elapsed Time: 0:00:02 ETA:   0:05:51
  0% (11 of 1563) |                      | Elapsed Time: 0:00:02 ETA:   0:05:54
  0% (12 of 1563) |                      | Elapsed Time: 0:00:02 ETA:   0:06:02
  0% (13 of 1563) |                      | Elapsed Time: 0:00:03 ETA:   0:06:05
  0% (14 of 1563) |                      | Elapsed Time: 0:00:03 ETA:   0:06:11
  0% (15 of 1563) |                     

2024-09-11 10:34:46,468 - Eval - INFO - Avg accuracy Top 1: 71.633077 Avg accuracy Top 5: 90.463052 on validation Dataset
71.63307741522712


---
## 3. Create a quantization simulation model and determine quantized accuracy

## Fold Batch Normalization layers
Before we determine the simulated quantized accuracy using QuantizationSimModel, we will fold the BatchNormalization (BN) layers in the model. These layers get folded into adjacent Convolutional layers. The BN layers that cannot be folded are left as they are.

**Why do we need to this?**
On quantized runtimes (like TFLite, SnapDragon Neural Processing SDK, etc.), it is a common practice to fold the BN layers. Doing so, results in an inferences/sec speedup since unnecessary computation is avoided. Now from a floating point compute perspective, a BN-folded model is mathematically equivalent to a model with BN layers from an inference perspective, and produces the same accuracy. However, folding the BN layers can increase the range of the tensor values for the weight parameters of the adjacent layers. And this can have a negative impact on the quantized accuracy of the model (especially when using INT8 or lower precision). So, we want to simulate that on-target behavior by doing BN folding here.

The following code calls AIMET to fold the BN layers in-place on the given model

In [9]:
from aimet_torch.batch_norm_fold import fold_all_batch_norms

bn_pairs = fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

print(len(bn_pairs))

2024-09-11 11:02:30,145 - BatchNormFolding - INFO - 0 BatchNorms' weights got converted
52


---
## Create Quantization Sim Model
Now we use AIMET to create a QuantizationSimModel. This basically means that AIMET will insert fake quantization ops in the model graph and will configure them.
A few of the parameters are explained here
- **quant_scheme**: We set this to "QuantScheme.post_training_tf_enhanced"
    - Supported options are 'tf_enhanced' or 'tf' or using Quant Scheme Enum QuantScheme.post_training_tf or QuantScheme.post_training_tf_enhanced
- **default_output_bw**: Setting this to 8, essentially means that we are asking AIMET to perform all activation quantizations in the model using integer 8-bit precision
- **default_param_bw**: Setting this to 8, essentially means that we are asking AIMET to perform all parameter quantizations in the model using integer 8-bit precision
- **num_batches**: The number of batches used to evaluate the model while calculating the quantization encodings.Number of batches to use for computing encodings. Only 5 batches are used here to speed up the process. In addition, the number of images in these 5 batches should be sufficient for compute encodings
- **rounding_mode**: The rounding mode used for quantization. There are two possible choices here - 'nearest' or 'stochastic' We will use "nearest."

There are other parameters that are set to default values in this example. Please check the AIMET API documentation of QuantizationSimModel to see reference documentation for all the parameters.

In [10]:
from aimet_common.defs import QuantScheme
from aimet_torch.quantsim import QuantizationSimModel

dummy_input = torch.rand(1, 3, 224, 224)    # Shape for eapch ImageNet sample is (3 channels) x (224 height) x (224 width)
if use_cuda:
    dummy_input = dummy_input.cuda()

sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

2024-09-11 11:02:31,920 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-09-11 11:02:31,971 - Quant - INFO - Unsupported op type Squeeze
2024-09-11 11:02:31,972 - Quant - INFO - Unsupported op type Mean
2024-09-11 11:02:31,979 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


---
We can check the modifications AIMET has made to the model graph. One way is to print the model, and we can see that AIMET has added quantization wrapper layers. Note: use sim.model to access the modified PyTorch model. By default, AIMET creates a copy of the original model prior to modifying it. There is a parameter to override this behavior.

In [11]:
print(sim.model)

GraphModule(
  (features): Module(
    (0): Module(
      (0): StaticGridQuantWrapper(
        (_module_to_wrap): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (1): Identity()
      (2): StaticGridQuantWrapper(
        (_module_to_wrap): ReLU6(inplace=True)
      )
    )
    (1): Module(
      (conv): Module(
        (0): Module(
          (0): StaticGridQuantWrapper(
            (_module_to_wrap): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
          )
          (1): Identity()
          (2): StaticGridQuantWrapper(
            (_module_to_wrap): ReLU6(inplace=True)
          )
        )
        (1): StaticGridQuantWrapper(
          (_module_to_wrap): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): Identity()
      )
    )
    (2): Module(
      (conv): Module(
        (0): Module(
          (0): StaticGridQuantWrapper(
            (_module_to_wrap): Conv2d(16, 96, kernel_size=(1, 1), stride=(1

---
We can also check how AIMET has configured the added fake quantization nodes, which AIMET refers to as 'quantizers'. You can see this by printing the sim object.

In [12]:
print(sim)

-------------------------
Quantized Model Report
-------------------------
----------------------------------------------------------
Layer: features.0.0
  Input[0]: bw=8, encoding-present=False
  -------
  Param[weight]: bw=8, encoding-present=False
  -------
  Param[bias]: Not quantized
  -------
  Output[0]: Not quantized
  -------
----------------------------------------------------------
Layer: features.0.2
  Input[0]: Not quantized
  -------
  Output[0]: bw=8, encoding-present=False
  -------
----------------------------------------------------------
Layer: features.1.conv.0.0
  Input[0]: Not quantized
  -------
  Param[weight]: bw=8, encoding-present=False
  -------
  Param[bias]: Not quantized
  -------
  Output[0]: Not quantized
  -------
----------------------------------------------------------
Layer: features.1.conv.0.2
  Input[0]: Not quantized
  -------
  Output[0]: bw=8, encoding-present=False
  -------
----------------------------------------------------------
Layer: fe

---
Even though AIMET has added 'quantizer' nodes to the model graph but the model is not ready to be used yet. Before we can use the sim model for inference or training, we need to find appropriate scale/offset quantization parameters for each 'quantizer' node. For activation quantization nodes, we need to pass unlabeled data samples through the model to collect range statistics which will then let AIMET calculate appropriate scale/offset quantization parameters. This process is sometimes referred to as calibration. AIMET simply refers to it as 'computing encodings'.

So we create a routine to pass unlabeled data samples through the model. This should be fairly simple - use the existing train or validation data loader to extract some samples and pass them to the model. We don't need to compute any loss metric etc. So we can just ignore the model output for this purpose. A few pointers regarding the data samples

In practice, we need a very small percentage of the overall data samples for computing encodings. For example, the training dataset for ImageNet has 1M samples. For computing encodings we only need 500 or 1000 samples.
It may be beneficial if the samples used for computing encoding are well distributed. It's not necessary that all classes need to be covered etc. since we are only looking at the range of values at every layer activation. However, we definitely want to avoid an extreme scenario like all 'dark' or 'light' samples are used - e.g. only using pictures captured at night might not give ideal results.
The following shows an example of a routine that passes unlabeled samples through the model for computing encodings. This routine can be written in many different ways, this is just an example.

In [13]:
def pass_calibration_data(sim_model, use_cuda):
    data_loader = ImageNetDataPipeline.get_val_dataloader()
    batch_size = data_loader.batch_size

    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    samples = 1000

    batch_cntr = 0
    with torch.no_grad():
        for input_data, target_data in data_loader:

            inputs_batch = input_data.to(device)
            sim_model(inputs_batch)

            batch_cntr += 1
            if (batch_cntr * batch_size) > samples:
                break

---
Now we call AIMET to use the above routine to pass data through the model and then subsequently compute the quantization encodings. Encodings here refer to scale/offset quantization parameters.

In [14]:
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

2024-09-11 11:02:32,267 - Dataloader - INFO - Dataset consists of 50000 images in 1000 classes


---
Now the QuantizationSim model is ready to be used for inference or training. First we can pass this model to the same evaluation routine we used before. The evaluation routine will now give us a simulated quantized accuracy score for INT8 quantization instead of the FP32 accuracy score we saw before.

In [15]:
accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

2024-09-11 11:02:50,504 - Dataloader - INFO - Dataset consists of 50000 images in 1000 classes
2024-09-11 11:02:50,509 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-09-11 11:02:50,510 - Eval - INFO - Evaluating nn.Module for 1563 iterations with batch_size 32


  0% (0 of 1563) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (2 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:03:40
  0% (3 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:04:44
  0% (4 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:06:08
  0% (5 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:06:45
  0% (6 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:07:40
  0% (7 of 1563) |                       | Elapsed Time: 0:00:02 ETA:   0:07:29
  0% (8 of 1563) |                       | Elapsed Time: 0:00:02 ETA:   0:07:22
  0% (9 of 1563) |                       | Elapsed Time: 0:00:02 ETA:   0:07:31
  0% (10 of 1563) |                      | Elapsed Time: 0:00:02 ETA:   0:07:28
  0% (11 of 1563) |                      | Elapsed Time: 0:00:03 ETA:   0:07:25
  0% (12 of 1563) |                      | Elapsed Time: 0:00:03 ETA:   0:07:27
  0% (13 of 1563) |                     

2024-09-11 11:08:10,078 - Eval - INFO - Avg accuracy Top 1: 68.570058 Avg accuracy Top 5: 88.497681 on validation Dataset
68.57005758157389


---
## 4. 1 Cross Layer Equalization

The next cell performs cross-layer equalization on the model. As noted before, the function folds batch norms, applies cross-layer scaling, and then folds high biases.

**Note:** Interestingly, CLE needs BN statistics for its procedure. If a BN folded model is provided, CLE will run the CLS (cross-layer scaling) optimization step but will skip the HBA (high-bias absorption) step. To avoid this, we simply load the original model again before running CLE.

**Note:** CLE equalizes the model in-place

In [16]:
model = copy.deepcopy(orig_model)
model = prepare_model(model)

use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.to(torch.device('cuda'))

2024-09-11 11:08:10,149 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.3.module_add} 
2024-09-11 11:08:10,150 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.5.module_add_1} 
2024-09-11 11:08:10,151 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.6.module_add_2} 
2024-09-11 11:08:10,151 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.8.module_add_3} 
2024-09-11 11:08:10,152 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.9.module_add_4} 
2024-09-11 11:08:10,153 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.10.module_add_5} 
2024-09-11 11:08:10,153 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.12.module_add_6} 
2024-09-11 11:08:10,154 - ModelPreparer - INFO - Functional         : Adding new module for node: {features.13.module_add_7} 

In [17]:
from aimet_torch.cross_layer_equalization import equalize_model

equalize_model(model, input_shapes=(1, 3, 224, 224))

2024-09-11 11:08:12,402 - BatchNormFolding - INFO - 0 BatchNorms' weights got converted


---
Now, we can determine the simulated quantized accuracy of the equalized model. We again create a simulation model like before and evaluate to determine simulated quantized accuracy.

In [18]:
sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

2024-09-11 11:08:18,318 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-09-11 11:08:18,364 - Quant - INFO - Unsupported op type Squeeze
2024-09-11 11:08:18,365 - Quant - INFO - Unsupported op type Mean
2024-09-11 11:08:18,375 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
2024-09-11 11:08:18,580 - Dataloader - INFO - Dataset consists of 50000 images in 1000 classes
2024-09-11 11:08:28,758 - Dataloader - INFO - Dataset consists of 50000 images in 1000 classes
2024-09-11 11:08:28,764 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-09-11 11:08:28,765 - Eval - INFO - Evaluating nn.Module for 1563 iterations with batch_size 32


  0% (0 of 1563) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (2 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:03:46
  0% (3 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:04:40
  0% (4 of 1563) |                       | Elapsed Time: 0:00:00 ETA:   0:05:59
  0% (5 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:06:30
  0% (6 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:07:26
  0% (7 of 1563) |                       | Elapsed Time: 0:00:01 ETA:   0:07:21
  0% (8 of 1563) |                       | Elapsed Time: 0:00:02 ETA:   0:07:19
  0% (9 of 1563) |                       | Elapsed Time: 0:00:02 ETA:   0:07:32
  0% (10 of 1563) |                      | Elapsed Time: 0:00:02 ETA:   0:07:28
  0% (11 of 1563) |                      | Elapsed Time: 0:00:03 ETA:   0:07:25
  0% (12 of 1563) |                      | Elapsed Time: 0:00:03 ETA:   0:07:24
  0% (13 of 1563) |                     

2024-09-11 11:13:56,111 - Eval - INFO - Avg accuracy Top 1: 70.375480 Avg accuracy Top 5: 89.577335 on validation Dataset
70.37547984644914


---
## 4. 2 Bias Correction

This section shows how we can apply AIMET Bias Correction on top of the already equalized model from the previous step. Bias correction under the hood uses a reference FP32 model and a QuantizationSimModel to perform its procedure. More details are explained in the AIMET User Guide documentation.

For the correct_bias API, we pass the following parameters

- **num_quant_samples**: Number of samples used for computing encodings. We are setting this to a low number here to speed up execution. A typical number would be 500-1000.
- **num_bias_correct_samples**: Number of samples used for bias correction. We are setting this to a low number here to speed up execution. A typical number would be 1000-2000.
- **data_loader**: BC uses unlabeled data samples from this data loader.

In [19]:
from aimet_torch.quantsim import QuantParams
from aimet_torch.bias_correction import correct_bias

data_loader = ImageNetDataPipeline.get_val_dataloader()

bc_params = QuantParams(weight_bw=8, act_bw=8, round_mode="nearest",
                        quant_scheme=QuantScheme.post_training_tf_enhanced)

correct_bias(model, bc_params, num_quant_samples=16,
             data_loader=data_loader, num_bias_correct_samples=16)

2024-09-10 13:01:26,630 - Dataloader - INFO - Dataset consists of 10000 images in 1000 classes


2024-09-10 13:01:29,976 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-09-10 13:01:30,008 - Quant - INFO - Unsupported op type Squeeze
2024-09-10 13:01:30,008 - Quant - INFO - Unsupported op type Mean
2024-09-10 13:01:30,013 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
2024-09-10 13:01:31,033 - Quant - INFO - Correcting layer features.0.0 using Empirical Bias Correction
2024-09-10 13:01:32,184 - Quant - INFO - Corrected bias for the layer
2024-09-10 13:01:32,188 - Quant - INFO - Correcting layer features.1.conv.0.0 using Empirical Bias Correction
2024-09-10 13:01:33,410 - Quant - INFO - Corrected bias for the layer
2024-09-10 13:01:33,415 - Quant - INFO - Correcting layer features.1.conv.1 using Empirical Bias Correction
2024-09-10 13:01:34,575 - Quant - INFO - Corrected bias for the layer
2024-09-10 1

---
Now, we can determine the simulated quantized accuracy of the bias-corrected model. We again create a simulation model like before and evaluate to determine simulated quantized accuracy.

In [20]:
sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(accuracy)

2024-09-10 13:02:35,447 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json
2024-09-10 13:02:35,494 - Quant - INFO - Unsupported op type Squeeze
2024-09-10 13:02:35,495 - Quant - INFO - Unsupported op type Mean
2024-09-10 13:02:35,501 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
2024-09-10 13:02:36,343 - Dataloader - INFO - Dataset consists of 10000 images in 1000 classes
2024-09-10 13:02:54,776 - Dataloader - INFO - Dataset consists of 10000 images in 1000 classes
2024-09-10 13:02:54,778 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2024-09-10 13:02:54,778 - Eval - INFO - Evaluating nn.Module for 313 iterations with batch_size 32


  0% (0 of 313) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  0% (2 of 313) |                        | Elapsed Time: 0:00:00 ETA:   0:00:29
  0% (3 of 313) |                        | Elapsed Time: 0:00:00 ETA:   0:00:39
  1% (4 of 313) |                        | Elapsed Time: 0:00:00 ETA:   0:00:43
  1% (5 of 313) |                        | Elapsed Time: 0:00:00 ETA:   0:00:46
  1% (6 of 313) |                        | Elapsed Time: 0:00:00 ETA:   0:00:49
  2% (7 of 313) |                        | Elapsed Time: 0:00:01 ETA:   0:00:50
  2% (8 of 313) |                        | Elapsed Time: 0:00:01 ETA:   0:00:51
  2% (9 of 313) |                        | Elapsed Time: 0:00:01 ETA:   0:00:51
  3% (10 of 313) |                       | Elapsed Time: 0:00:01 ETA:   0:00:52
  3% (11 of 313) |                       | Elapsed Time: 0:00:01 ETA:   0:00:52
  3% (12 of 313) |                       | Elapsed Time: 0:00:02 ETA:   0:00:53
  4% (13 of 313) |                      

2024-09-10 13:03:57,084 - Eval - INFO - Avg accuracy Top 1: 70.956470 Avg accuracy Top 5: 89.586661 on validation Dataset
70.9564696485623


---
Depending on your settings you may have observed a slight gain in accuracy after applying CLE ad BC. Ofcourse, this was just an example. Please try this against the model of your choice and play with the number of samples to get the best results.

Now the next step would be to take this model to target. For this purpose, we need to export the model with the updated weights without the fake quant ops. And also to export the encodings (scale/offset quantization parameters). AIMET QuantizationSimModel provides an export API for this purpose.

In [21]:
os.makedirs('./output/', exist_ok=True)
dummy_input = dummy_input.cpu()
sim.export(path='./output/', filename_prefix='resnet18_after_cle_bc', dummy_input=dummy_input)

2024-09-10 13:03:57,095 - Quant - WARNING - Exporting encodings to yaml will be deprecated in a future release. Ensure that your code can work with the exported files ending in ".encodings" which are saved using json format. For the time being, if yaml export is needed, set aimet_common.utils.SAVE_TO_YAML to True.


[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W shape_type_inference.cpp:1973] Warning: The shape inference of aimet_torch::CustomMarker type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W s

2024-09-10 13:03:58,678 - Utils - INFO - successfully created onnx model with 99/100 node names updated
2024-09-10 13:03:58,717 - Quant - WARNING - number of input quantizers: 1 available for layer: features.0.0 doesn't match with number of input tensors: 3
2024-09-10 13:03:58,737 - Quant - INFO - Layers excluded from quantization: []


---
## Summary

Hope this notebook was useful for you to understand how to use AIMET for performing Cross Layer Equalization (CLE) and Bias Correction (BC).

Few additional resources
- Refer to the AIMET API docs to know more details of the APIs and optional parameters
- Refer to the other example notebooks to understand how to use AIMET post-training quantization techniques and QAT techniques